# Vibration Isolation

#### Preliminaries

We have to import the mathematical functions that will be used in the following. Also, we want to define a helper function to properly format our results.

In [5]:
from math import atan2, cos, exp, pi, sin, sqrt, tan
def plvu(label,  value, units=""):
    print("%40s: %10g %s"%(label, value, units))

#### Problem's data 

The data of our problem is as follows $\def\T{\text{TR}}$

In [6]:
m0 = 7100.0 # kg
p0 = 2400.0 # N
f0 =   20.0 # Hz
pr =  300.0 # N
z  = 8./100 # Damping ratio
plvu("Mass of the machine", m0, "kg")
plvu("Unbalanced force", p0, "N")
plvu("Frequency of unbalanced force", f0, "Hz")
plvu("Max value of transmitted s-s force", pr, "N")
plvu("Damping ratio in second part of problem", z*100, "%")

                     Mass of the machine:       7100 kg
                        Unbalanced force:       2400 N
           Frequency of unbalanced force:         20 Hz
      Max value of transmitted s-s force:        300 N
 Damping ratio in second part of problem:          8 %


## Undamped system

The requested transmissibility ratio is $\T=p_{ss}/p_0$, while the transmissibility ratio of an undamped system is $1/(\beta^2-1)$, where $\beta$ is the frequency ratio, $\beta=\omega_0/\omega_n$, with $\omega_0$ and $\omega_n$ being respectively the excitation frequency and the natural frequency of the system.

At first we compute the requested transmissibility ratio and the circular frequency of the excitation, as the problem gave it in terms of Hertz.

In [7]:
TR = pr/p0
plvu("The required transmissibility ratio", TR)
w0 = 2*pi*f0
W0 = w0**2
plvu("Excitation circular frequency, squared", W0,"(rad/s)^2.")

     The required transmissibility ratio:      0.125 
  Excitation circular frequency, squared:    15791.4 (rad/s)^2.


Because the system transmissibility must be no greater than the requested one we have:

\begin{align*}
  \T &\ge \frac{1}{\beta^2-1} = \frac{\omega_n^2}{\omega_0^2-\omega_n^2} \Rightarrow\\
  \omega_n^2 &\le (\omega_0^2-\omega_n^2)\,\T\Rightarrow\\
  \omega_n^2 &\le \frac{\T}{1+\T}\,\omega_0^2.
\end{align*}

In [8]:
Wn = W0*TR/(1+TR)
plvu("System circular frequency, squared", Wn, "(rad/s)^2")

      System circular frequency, squared:     1754.6 (rad/s)^2


The natural frequency is related to the mass: $\omega_n^2=k/m$. Substituting 
in the previous disequation and solving for $k$ 

$$k \le  \frac{\T}{1+\T}\,m\,\omega_0^2=k_\text{max}$$

we obtain $k_\text{max}$, the maximum stiffness for which the transmisibility ratio is not greater than the requested one.

In [9]:
k0 = m0*Wn
plvu("Maximum stiffness of support", k0/1E6, "kN/mm")

            Maximum stiffness of support:    12.4576 kN/mm


While we are at it, we compute also the minimum static displacement of the system,

$$ \Delta_\text{st} \ge \frac{m\,g}{k_\text{max}}.$$

In [10]:
plvu("Minimum static displacement", m0*9.81/k0*1000, "mm")

             Minimum static displacement:    5.59103 mm


## Damped system

The transmissibility ratio is

$$\T=\sqrt\frac{1+(2\zeta\beta)^2}{(1-\beta^2)^2+(2\zeta\beta)^2}.$$

In the second part of our problem, $\zeta=0.08$ is assigned and $\T$ is known, so the above expression is a function of $\beta$, the frequency ratio. We need to know for which value of $\beta$ the above equation holds.

Squaring both members, taking all factors on the same side of the equal sign, expanding, collecting $\beta^2$ and eventually dividing by $\T^2$ we have

$${\beta^2}^2 + 2\left(
2\zeta^2\frac{\T^2-1}{\T^2}-1
\right)\beta^2 + \frac{\T^2-1}{\T^2} = 0,$$

an algebraic equation of second order in $\beta^2$.

With $a=2\zeta^2\frac{\T^2-1}{\T^2}-1$ and $b=\frac{\T^2-1}{\T^2}$

In [11]:
b = (TR**2-1)/TR**2
a = 2*z*z*b-1
plvu("Coefficient of linear term", a)
plvu("Constant coefficient", b)

              Coefficient of linear term:    -1.8064 
                    Constant coefficient:        -63 


the positive root is given by

$$\beta^2 = \sqrt{a^2-b}-a.$$

In [12]:
b2 = sqrt(a*a-b)-a
plvu("Minimum value of squared freq.ratio, damped system", b2)

Minimum value of squared freq.ratio, damped system:    9.94661 


We have already installed the elastic support, as specified from the undamped design, we add a damper and the requested minimum frequency ratio increases... How we can increase the frequency ratio?

Decreasing the natural frequency of the system, but the springs are already in place and the only thing that we can do is to change the mass of the system, in particular we decrease the frequency by increasing the mass and this is OK, because we can always add ballast to our system while, on the other hand, it would be, in general, impossible to reduce the system mass...

The  max value of the natural frequency is given by 

$$\beta^2=\omega^2_0/\omega^2_n\Rightarrow \omega_n^2=\omega^2_0/\beta^2$$

In [13]:
Wnmx = W0/b2
plvu("Max admissible value of squared natural frequency", Wnmx, "(rad/s)^2")

Max admissible value of squared natural frequency:    1587.61 (rad/s)^2


 Eventually, the minimum mass of the damped system is
 
 $$m=k/\omega_n^2.$$
 
We compute also the minimum mass of the ballast, the percent mass increment and the new static displacement.

In [14]:
mmn = k0/Wnmx
plvu("Minimum mass of the damped system", mmn, "kg")
plvu("Minimum mass of ballast", mmn-m0, "kg")
plvu("Mass increment", (mmn-m0)*100/m0, "%")
plvu("Minimum static displacement, d.s.", 9.81*mmn/k0*1000, "mm")

       Minimum mass of the damped system:    7846.77 kg
                 Minimum mass of ballast:    746.773 kg
                          Mass increment:    10.5179 %
       Minimum static displacement, d.s.:    6.17909 mm


In [15]:
# The following incantation gives a proper style to this notebook
from IPython.display import display, Latex, HTML
HTML(open("00_custom.css", "r").read())